In [1]:
import heapq

class EightPuzzle:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.n = 3  #grille 3x3
        
    # Trouve la position de la case vide 
    def find_empty(self, state):
        for i in range(self.n):
            for j in range(self.n):
                if state[i][j] == 0: #0 représente la case vide
                    return i, j
    
    # Vérifie si le puzzle est résolu
    def is_goal(self, state):
        return state == self.goal_state
    
    # Retourne les actions possibles 
    def actions(self, state):
        row, col = self.find_empty(state)#trouve la case vide
        moves = []
        if row > 0: moves.append('up')  
        if row < self.n - 1: moves.append('down') 
        if col > 0: moves.append('left') 
        if col < self.n - 1: moves.append('right')  
        return moves

    # Apllique un déplacement
    def result(self, state, action):
        row, col = self.find_empty(state)
        new_state = [list(row) for row in state] 
        
        if action == 'up':
            new_state[row][col], new_state[row - 1][col] = new_state[row - 1][col], new_state[row][col]
        elif action == 'down':
            new_state[row][col], new_state[row + 1][col] = new_state[row + 1][col], new_state[row][col]
        elif action == 'left':
            new_state[row][col], new_state[row][col - 1] = new_state[row][col - 1], new_state[row][col]
        elif action == 'right':
            new_state[row][col], new_state[row][col + 1] = new_state[row][col + 1], new_state[row][col]
        
        return new_state
    
    # Génère le chemin des actions effectuées pour atteindre l'état but
    def print_path(self, came_from, current):
        path = []
        while current in came_from:
            current, action = came_from[current]
            path.append(action)
        return path[::-1]  # Retourne le chemin dans le bon ordre (du début à la fin)


    # Algorithme de recherche uniforme par cout
    def recherche_uniforme(self):
        frontiere = []  
        heapq.heappush(frontiere, (0, self.initial_state))  # Ajouter l'état initial avec un coût de 0
        vient_de = {}  # Dictionnaire pour suivre le chemin emprunté
        cout_actuel = {tuple(map(tuple, self.initial_state)): 0}  # Dictionnaire des coûts pour chaque état
        noeuds_visites = 0 
        
        while frontiere:
            cout, courant = heapq.heappop(frontiere)  # Extraire l'état avec le coût le plus bas
            noeuds_visites += 1 
            
            # vérifie que l'état est le bon
            if self.is_goal(courant):
                chemin = self.print_path(vient_de, tuple(map(tuple, courant)))  # Générer le chemin des actions
                return len(chemin), noeuds_visites,chemin
            
            # Parcourir toutes les actions possibles à partir de l'état courant
            for action in self.actions(courant):
                nouvel_etat = self.result(courant, action)  # Calculer le nouvel état après l'action
                profondeur = cout + 1  # Chaque déplacement coûte 1
                
                # Si le nouvel état n'a pas encore été visité ou si un chemin moins coûteux est trouvé
                if tuple(map(tuple, nouvel_etat)) not in cout_actuel or profondeur < cout_actuel[tuple(map(tuple, nouvel_etat))]:
                    cout_actuel[tuple(map(tuple, nouvel_etat))] = profondeur 
                    heapq.heappush(frontiere, (profondeur, nouvel_etat))  
                    vient_de[tuple(map(tuple, nouvel_etat))] = (tuple(map(tuple, courant)), action)  # Garder une trace de l'action et de l'état courant
        
        return None  

    # Algorithme de recherche BFS avec heuristique
    def bfs_with_heuristic(self, heuristique):
        frontiere = []
        heapq.heappush(frontiere, (heuristique(self.initial_state), 0, self.initial_state))  # (priorité, profondeur, état)
        vient_de = {}  # Pour suivre le chemin
        cout_actuel = {tuple(map(tuple, self.initial_state)): 0}  # Initialiser avec l'état initial à un coût de 0
        noeuds_visites = 0 

        while frontiere:
            priorite, profondeur, courant = heapq.heappop(frontiere)  # Extraire l'état avec la plus petite priorité
            noeuds_visites += 1

            # Si l'état but est atteint
            if self.is_goal(courant):
                chemin = self.print_path(vient_de, tuple(map(tuple, courant)))
                return len(chemin), noeuds_visites, chemin 
            
            
            # Parcourir toutes les actions possibles
            for action in self.actions(courant):
                nouvel_etat = self.result(courant, action)
                etat_tuple = tuple(map(tuple, nouvel_etat))
                nouveau_cout = profondeur + 1  # Calculer le coût pour le nouvel état

                # Si l'état n'a pas encore été visité ou si un chemin moins coûteux est trouvé
                if etat_tuple not in cout_actuel or nouveau_cout < cout_actuel[etat_tuple]:
                    cout_actuel[etat_tuple] = nouveau_cout  
                    vient_de[etat_tuple] = (tuple(map(tuple, courant)), action)  # Garder une trace de l'action et de l'état courant
                    # Calculer la priorité a l'heuristique
                    priorite = heuristique(nouvel_etat)
                    heapq.heappush(frontiere, (priorite, nouveau_cout, nouvel_etat))

        return None  


    # Heuristique 1: Comptage des tuiles mal placées
    def h1(self, state):
        misplaced = 0
        for i in range(self.n):
            for j in range(self.n):
                if state[i][j] != 0 and state[i][j] != self.goal_state[i][j]:
                    misplaced += 1
        return misplaced

    # Heuristique 2: Distance de Manhattan
    def h2(self, state):
        total_distance = 0
        for i in range(self.n):
            for j in range(self.n):
                tile = state[i][j]
                if tile != 0:  # Ignorer la case vide
                    # Trouver la position de la tuile dans l'état objectif
                    for goal_row in range(self.n):
                        for goal_col in range(self.n):
                            if self.goal_state[goal_row][goal_col] == tile:
                                # Calculer la distance de Manhattan
                                total_distance += abs(goal_row - i) + abs(goal_col - j)
                                break 
        return total_distance
 
    # Heuristique 3: Compte des tuiles non dans leur ligne ou colonne
    def h3(self, state):
        count = 0
        for i in range(self.n):
            for j in range(self.n):
                tile = state[i][j]
                if tile != 0:  
                   
                    for goal_row in range(self.n):
                        for goal_col in range(self.n):
                            if self.goal_state[goal_row][goal_col] == tile:
                                
                                if goal_row != i:
                                    count += 1  # Pas dans la bonne ligne
                                if goal_col != j:
                                    count += 1  # Pas dans la bonne colonne
                                break  
        return count

    
    

    # Algorithme A* avec heuristique
    def a_star(self, heuristic):
        
        frontier = []  # Liste qui contiendra les états à explorer   
        heapq.heappush(frontier, (0, self.initial_state))   
        vient_de = {}
        # Coût total jusqu'à chaque état, initialisé avec l'état initial à un coût de 0
        cout_actuel = {tuple(map(tuple, self.initial_state)): 0}
        noeuds_visites = 0

        
        while frontier:
           
            _, courant = heapq.heappop(frontier)   # Extraction de l'état avec la plus petite priorité
            
            noeuds_visites += 1 # Incrémenter le nombre de noeuds visités
            
            if self.is_goal(courant):
                
                chemin = self.print_path(vient_de, tuple(map(tuple, courant)))
                return len(chemin), noeuds_visites, chemin
            
            for action in self.actions(courant):
                # Calculer le nouvel état résultant de l'application de l'action
                nouvel_etat = self.result(courant, action)
                # Coût du nouvel état, 1 étant le coût fixe pour chaque action
                nouveau_cout = cout_actuel[tuple(map(tuple, courant))] + 1  
                # Convertir le nouvel état en un tuple pour un accès facile dans le dictionnaire
                etat = tuple(map(tuple, nouvel_etat))

                # Vérifier si l'état n'a pas encore été visité ou si on a trouvé un chemin moins coûteux
                if etat not in cout_actuel or nouveau_cout < cout_actuel[etat]:
                    
                    cout_actuel[etat] = nouveau_cout  
                    #print(nouveau_cout)
                    
                    priority = nouveau_cout + heuristic(nouvel_etat)
                    # Ajouter le nouvel état à la frontière avec sa priorité
                    heapq.heappush(frontier, (priority, nouvel_etat))
                    # Garder trace de l'état précédent et de l'action qui a conduit à cet état
                    vient_de[etat] = (tuple(map(tuple, courant)), action)    
        return None  
    


In [2]:

initial_state = [
    [7, 2, 4],
    [5, 0, 6],  
    [8, 3, 1]
            ]

goal_state = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]
            ]

puzzle = EightPuzzle(initial_state, goal_state)

# Uniform Cost Search
steps_ucs, nodes_ucs, chemin = puzzle.recherche_uniforme()
print(f"Uniform Cost Search:\nétapes =", steps_ucs, "Noeud visité =", nodes_ucs, "\nchemin:", chemin)


Uniform Cost Search:
étapes = 26 Noeud visité = 162241 
chemin: ['left', 'up', 'right', 'down', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left']


In [3]:
print("BFS with misplaced tiles:")
path_length, nodes_expanded, chemin = puzzle.bfs_with_heuristic(puzzle.h1)
print("étape:", path_length," noeud visité: ",nodes_expanded, "\nchemin:", chemin)

print("\n------------------------------------------------------------------------------------------\n")

# Utiliser BFS avec heuristique h2 (Manhattan distance)
print("BFS Manhattan distance:")
path_length, nodes_expanded, chemin = puzzle.bfs_with_heuristic(puzzle.h2)
print("étape: ",path_length,"noeud visité:", nodes_expanded, "\nchemin:", chemin)

BFS with misplaced tiles:
étape: 54  noeud visité:  1001 
chemin: ['right', 'up', 'left', 'left', 'down', 'right', 'right', 'down', 'left', 'left', 'up', 'up', 'right', 'down', 'right', 'down', 'left', 'left', 'up', 'up', 'right', 'down', 'left', 'down', 'right', 'up', 'left', 'up', 'right', 'down', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'up', 'right', 'down', 'left', 'left', 'up', 'right', 'down', 'right', 'up', 'left', 'left']

------------------------------------------------------------------------------------------

BFS Manhattan distance:
étape:  60 noeud visité: 462 
chemin: ['down', 'right', 'up', 'up', 'left', 'left', 'down', 'right', 'down', 'left', 'up', 'up', 'right', 'down', 'left', 'down', 'right', 'right', 'up', 'left', 'down', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'up', 'right', 'down', 'left', 'down', 'right', 'up', 'left', 'down', 'right', 'up', 'up', 'left', 'down', 'right', 'down', 'left

In [4]:
steps_a_star_h1, nodes_a_star_h1, chemin = puzzle.a_star(puzzle.h1)
print("A* Misplaced tiles: \nétape =", steps_a_star_h1, "noeud visté = ",nodes_a_star_h1, "\nchemin:", chemin)

print("\n------------------------------------------------------------------------------------------\n")

steps_a_star_h2, nodes_a_star_h2, chemin = puzzle.a_star(puzzle.h2)
print("A* Manhattan distance: \nétape = ", steps_a_star_h2, "noeud visté = ",nodes_a_star_h2, "\nchemin:", chemin)

A* Misplaced tiles: 
étape = 26 noeud visté =  33153 
chemin: ['left', 'up', 'right', 'down', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left']

------------------------------------------------------------------------------------------

A* Manhattan distance: 
étape =  26 noeud visté =  2034 
chemin: ['left', 'up', 'right', 'down', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left']


h2 renvoie le meme nombre d'étape pour atteindre l'objectif que h1 mais h2 est plus performant car il explore moins de noeud que h1 pour obtenir ce resultat

In [5]:
print("BFS h3:")
path_length, nodes_expanded, chemin = puzzle.bfs_with_heuristic(puzzle.h3)
print("étape:", path_length, "noeud visté: ",nodes_expanded, "\nchemin:", chemin)

BFS h3:
étape: 38 noeud visté:  331 
chemin: ['down', 'right', 'up', 'up', 'left', 'left', 'down', 'right', 'down', 'left', 'up', 'up', 'right', 'down', 'left', 'down', 'right', 'up', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'up', 'left', 'up', 'right', 'down', 'left', 'down', 'right', 'up', 'up', 'left']


UCS trouve le chemin le plus rapide pour arrivé a l'état recherché mais il explore énormément de noeud ce qui n'est pas très optimale

BFS pour h1 et h2 trouve un chemin pour arrivé à l'état rechercher mais avec h1, l'algorithmes parcour beaucoup plus de noeud que avec h2 (meme si au final, h1 trouve une meilleur solution que h2). Cela s'explique sans doute car la distance de manathan permet d'obtenir plus de précisions par rapport à deux tuile alors que l'heurestic des tuiles mal placé ne prend pas vraiment en compte quelle tuile est "analysé"

Pour A*, c'est la meme conclusions que pour le BFS sauf que A* est un peu plus performant que BFS. Cela peut s'expliquer par le fait que BFS utilise simplement la profondeur et l'heurestic donné pour calculer la priorité alors que A* gère bien mieux la priorité en ajoutant le coup de l'étape précédente.

pour BFS avec h3, cela renvoie un chemin acceptable mais explore plus de noeud qu'avec h2. Cela montre que prendre comme heuretic le fait d'additionner le nombre de colonne et de ligne mauvaise n'est pas une mauvaise idée mais la distance de manathan reste meilleure.
Ainsi, h3 est admissible car elle ne depasse pas la réalité mais h2 vaut plus le coup.